In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import ipyleaflet
from branca.colormap import linear
import branca.colormap as cm
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
#temp_by_city = pd.read_csv("GlobalLandTemperaturesByCity.csv")
temp_by_country = pd.read_csv("GlobalLandTemperaturesByCountry.csv")

In [3]:
temp_by_country = temp_by_country[~temp_by_country['Country'].isin(
    ['Denmark', 'Antarctica', 'France', 'Europe', 'Netherlands',
     'United Kingdom', 'Africa', 'South America'])]

temp_by_country = temp_by_country.replace(
   ['Denmark (Europe)', 'France (Europe)', 'Netherlands (Europe)', 'United Kingdom (Europe)', 'Åland', 'United States', 'Burma'],
   ['Denmark', 'France', 'Netherlands', 'United Kingdom', 'Aland', 'United States of America', 'Myanmar'])

In [4]:
def after_1880(dt):
    year = int(dt[:4])
    return year >= 1880

In [5]:
#keep only records after 1880, because some of the records in some regions are missing before 1880
temp_by_country = temp_by_country[temp_by_country.get("dt").apply(after_1880)]

In [6]:
temp_by_country

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
1634,1880-01-01,-1.928,1.186,Aland
1635,1880-02-01,-0.920,0.927,Aland
1636,1880-03-01,-0.838,0.671,Aland
1637,1880-04-01,2.111,0.514,Aland
1638,1880-05-01,6.393,0.727,Aland
...,...,...,...,...
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe
577460,2013-08-01,19.759,0.717,Zimbabwe


In [7]:
def season(dt):
    month = int(dt[5:7])
    if (month >= 3 and month <=5):
        return "Spring"
    if (month >= 6 and month <=8):
        return "Summer"
    if (month >= 9 and month <=11):
        return "Fall"
    return "Winter"

In [8]:
temp_by_country = temp_by_country.assign(Season = temp_by_country.get("dt").apply(season))

In [9]:
temp_by_country

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Season
1634,1880-01-01,-1.928,1.186,Aland,Winter
1635,1880-02-01,-0.920,0.927,Aland,Winter
1636,1880-03-01,-0.838,0.671,Aland,Spring
1637,1880-04-01,2.111,0.514,Aland,Spring
1638,1880-05-01,6.393,0.727,Aland,Spring
...,...,...,...,...,...
577457,2013-05-01,19.059,1.022,Zimbabwe,Spring
577458,2013-06-01,17.613,0.473,Zimbabwe,Summer
577459,2013-07-01,17.000,0.453,Zimbabwe,Summer
577460,2013-08-01,19.759,0.717,Zimbabwe,Summer


In [10]:
temp_by_country_summer = temp_by_country[temp_by_country.get("Season") == "Summer"]
temp_by_country_winter = temp_by_country[temp_by_country.get("Season") == "Winter"]

In [11]:
def  dt_to_year(dt):
    return int(dt[0:4])

In [12]:
temp_by_country_summer = temp_by_country_summer.assign(dt_year_only = temp_by_country_summer.get('dt').apply(dt_to_year))

In [13]:
temp_by_country_winter = temp_by_country_winter.assign(dt_year_only = temp_by_country_winter.get('dt').apply(dt_to_year))

In [14]:
year_temp_summer = temp_by_country_summer.get(['dt_year_only','AverageTemperature','Country']).groupby(['dt_year_only','Country']).mean()

In [15]:
year_temp_summer

AverageTemperature
dt_year_only Country                           
1880         Afghanistan              25.386333
             Aland                    14.764667
             Albania                  21.316333
             Algeria                  32.344000
             American Samoa                 NaN
...                                         ...
2013         Virgin Islands           28.599333
             Western Sahara           27.612000
             Yemen                    31.499333
             Zambia                   19.285333
             Zimbabwe                 18.124000

[31280 rows x 1 columns]

In [16]:
year_temp_winter = temp_by_country_winter.get(['dt_year_only','AverageTemperature','Country']).groupby(['dt_year_only','Country']).mean().reset_index()
year_temp_summer = temp_by_country_summer.get(['dt_year_only','AverageTemperature','Country']).groupby(['dt_year_only','Country']).mean().reset_index()

In [17]:
geo_data = None
with open('custom.geo.json', 'r') as f:
    geo_data = json.load(f)
country_list = geo_data['features']
all_country_list = []
for one_country in country_list:
    one_country['id'] = one_country['properties']['sovereignt']
    all_country_list.append(one_country['properties']['sovereignt'])

In [18]:
temp_base = year_temp_winter[(year_temp_winter.get('dt_year_only')<1920) & (year_temp_winter.get('dt_year_only')>1900)].groupby('Country').mean()
temp_base = dict(temp_base.get('AverageTemperature'))

temp_base_s = year_temp_summer[(year_temp_summer.get('dt_year_only')<1920) & (year_temp_summer.get('dt_year_only')>1900)].groupby('Country').mean()
temp_base_s = dict(temp_base_s.get('AverageTemperature'))

In [19]:
def calculate_rolling(data_in):
    rolling_data = []
    rolling = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
    for i in range(len(data_in)):
        data_point = {}
        for k,v in data_in[i].items():
            num_rolling = 0
            val_rolling = 0
            for j in range(len(rolling)):
                roll = rolling[j]
                if not (i + roll <0 or i +roll >= len(data_in)):
                    try:
                        val_rolling += data_in[i + roll][k]
                        num_rolling += 1
                    except:
                        pass
            data_point[k] = val_rolling/num_rolling
        rolling_data.append(data_point)
    return rolling_data

In [20]:
datas = []
for i in range(133):
    temp_winter = year_temp_winter[(year_temp_winter.get('dt_year_only') == 1880 + i)]
    data = dict(temp_winter.set_index('Country').get('AverageTemperature'))
    data_new = {}
    for k,v in data.items():
        if (k not in temp_base.keys() or pd.isna(v)):
            data_new[k] = 0
        else:
            data_new[k] = v - temp_base[k]
            
    for oneC in all_country_list:
        if (oneC not in data_new.keys()):
            data_new[oneC] = 0
    datas.append(data_new)
datas = calculate_rolling(datas)

In [21]:
datas_s = []
for i in range(133):
    temp_s = year_temp_summer[(year_temp_summer.get('dt_year_only') == 1880 + i)]
    data = dict(temp_s.set_index('Country').get('AverageTemperature'))
    data_new = {}
    for k,v in data.items():
        if (k not in temp_base.keys() or pd.isna(v)):
            data_new[k] = 0
        else:
            data_new[k] = v - temp_base_s[k]
            
    for oneC in all_country_list:
        if (oneC not in data_new.keys()):
            data_new[oneC] = 0
    datas_s.append(data_new)
datas_s = calculate_rolling(datas_s)

In [22]:
int_range = widgets.IntSlider(
    value=1880,
    min=1880,
    max=2011,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
def on_value_change(change):
    with output2:
        #m.remove_layer(layers[change['old'] - 1880])
        m.add_layer(layers[change['new'] - 1880])
        m.remove_layer(m.layers[1])
        #print(change)

In [23]:
color_map = cm.LinearColormap([(51,160,255),(153,222,255), 'white', (255,208,51), 'orange', 'red'],
                  index=[0, 0.32,0.325, 0.33, 0.66, 1])

In [24]:
color_map

In [25]:
linear.YlOrRd_04

In [26]:
int_range_s = widgets.IntSlider(
    value=1880,
    min=1880,
    max=2011,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
def on_value_change_s(change):
    with output2_s:
        #m.remove_layer(layers[change['old'] - 1880])
        m_s.add_layer(layers_s[change['new'] - 1880])
        m_s.remove_layer(m_s.layers[1])
        #print(change)

In [27]:
layers = []
for i in range(133):
    layer = ipyleaflet.Choropleth(
        geo_data=geo_data,
        choro_data=datas[i],
        colormap=color_map,
        border_color='black',
        style={'fillOpacity': 1, 'dashArray': '5, 5'},
        value_min = -1, value_max = 2)
    layers.append(layer);


In [28]:
m = ipyleaflet.Map(center = (43,-100),zoom = 2)
m.add_layer(layers[0])
m

Map(center=[43, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [29]:
output2 = widgets.Output()
display(int_range, output2)
int_range.observe(on_value_change, names='value')

IntSlider(value=1880, description='Test:', max=2011, min=1880)

Output()

In [30]:
layers_s = []
for i in range(133):
    layer = ipyleaflet.Choropleth(
        geo_data=geo_data,
        choro_data=datas_s[i],
        colormap=color_map,
        border_color='black',
        style={'fillOpacity': 1, 'dashArray': '5, 5'},
        value_min = -1, value_max = 2)
    layers_s.append(layer);



In [31]:
m_s = ipyleaflet.Map(center = (43,-100),zoom = 2)
m_s.add_layer(layers_s[0])
m_s

Map(center=[43, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [32]:
output2_s = widgets.Output()
display(int_range_s, output2_s)
int_range_s.observe(on_value_change_s, names='value')

IntSlider(value=1880, description='Test:', max=2011, min=1880)

Output()

In [33]:
# m_b = ipyleaflet.Map(center = (43,-100),zoom = 2)
# control = ipyleaflet.SplitMapControl(left_layer=layers_s[0], right_layer=layers[0])
# m_b.add_control(control)

# m_b

In [34]:
# int_range_s = widgets.IntSlider(
#     value=1880,
#     min=1880,
#     max=2011,
#     step=1,
#     description='Test:',
#     disabled=False,
#     continuous_update=True,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d'
# )
# output2_s = widgets.Output()

# display(int_range_s, output2_s)

# def on_value_change_s(change):
#     with output2_s:
#         #m.remove_layer(layers[change['old'] - 1880])
#         control.left_layer = layers_s[change['new'] - 1880]
#         control.right_layer = layers[change['new'] - 1880]
#         #print(change)

# int_range_s.observe(on_value_change_s, names='value')

In [35]:
datas[132]

{'Afghanistan': 0.5088654970760236,
 'Aland': 0.3700263157894736,
 'Albania': 0.5873450292397658,
 'Algeria': 1.5990000000000002,
 'American Samoa': 0.7919356725146199,
 'Andorra': 0.775374269005848,
 'Angola': 0.3305292397660817,
 'Anguilla': 1.2848947368421026,
 'Antigua And Barbuda': 1.2703157894736836,
 'Argentina': 1.1536549707602315,
 'Armenia': 1.0895263157894741,
 'Aruba': 1.3231169590643284,
 'Asia': 1.2072456140350873,
 'Australia': 0.6889181286549722,
 'Austria': 0.9537309941520465,
 'Azerbaijan': 1.0263362573099413,
 'Bahamas': 0.779108187134503,
 'Bahrain': 1.0651520467836237,
 'Baker Island': 1.061538011695906,
 'Bangladesh': 0.9356023391812869,
 'Barbados': 1.3189590643274893,
 'Belarus': 0.7939824561403506,
 'Belgium': 0.8039415204678364,
 'Belize': 1.1782251461988302,
 'Benin': 1.4212748538011677,
 'Bhutan': 1.2629005847953214,
 'Bolivia': 0.8380701754385953,
 'Bonaire, Saint Eustatius And Saba': 1.4035555555555554,
 'Bosnia And Herzegovina': 0.8195555555555556,
 'Bots

In [36]:
datas = []
for i in range(133):
    temp_winter = year_temp_winter[(year_temp_winter.get('dt_year_only') == 1880 + i)]
    data = dict(temp_winter.set_index('Country').get('AverageTemperature'))
    data_new = {}
    for k,v in data.items():
        if (k not in temp_base.keys() or pd.isna(v)):
            data_new[k] = 0
        else:
            data_new[k] = v - temp_base[k]
            
    for oneC in all_country_list:
        if (oneC not in data_new.keys()):
            data_new[oneC] = 0
    datas.append(data_new)
datas[130]

{'Afghanistan': 2.037754385964913,
 'Aland': -4.040473684210527,
 'Albania': 1.0684561403508779,
 'Algeria': 3.9066666666666663,
 'American Samoa': 1.0328245614035083,
 'Andorra': -0.5934035087719298,
 'Angola': 0.7741403508771931,
 'Anguilla': 1.572561403508768,
 'Antigua And Barbuda': 1.6189824561403512,
 'Argentina': 1.137210526315787,
 'Armenia': 4.414526315789473,
 'Aruba': 1.7152280701754385,
 'Asia': 0.7272456140350876,
 'Australia': 0.6491403508771931,
 'Austria': -1.1014912280701754,
 'Azerbaijan': 3.540614035087719,
 'Bahamas': -1.3602807017543874,
 'Bahrain': 2.631263157894736,
 'Baker Island': 1.2446491228070187,
 'Bangladesh': 0.790824561403511,
 'Barbados': 1.7750701754386036,
 'Belarus': -2.5366842105263157,
 'Belgium': -2.308280701754386,
 'Belize': 0.07428070175438606,
 'Benin': 2.0650526315789435,
 'Bhutan': 1.1297894736842107,
 'Bolivia': 1.3157368421052595,
 'Bonaire, Saint Eustatius And Saba': 1.849333333333334,
 'Bosnia And Herzegovina': 0.22833333333333328,
 'Bot

In [37]:
cm.LinearColormap([(84,222,255), 'white', 'yellow', 'orange', 'red'],
                  index=[0, 0.28, 0.33, 0.66, 1])